Resumen: este bloque construye un DataFrame con features agregadas (conteos, sumas, medias, medianas, máximos) por cada token (mint), incluyendo cuántas transacciones “buy” y “sell”. El resultado es un único CSV (gran_chunk.csv) con todas estas variables, que luego se mergeará con tus datos de train/test.

In [1]:
import os
import pandas as pd

# Directorio donde se encuentran los archivos chunck
chunk_dir = r"D:\Desafio Kaggle\Desafios SolanaV2"

# Lista para guardar los DataFrames individuales de cada archivo chunk
all_chunks = []

# Iterar sobre los archivos chunck_1.csv a chunck_41.csv
for i in range(1, 42):
    file_path = os.path.join(chunk_dir, f"chunk_{i}.csv")
    df = pd.read_csv(file_path)
    all_chunks.append(df)

# Concatenar todos los DataFrames en uno solo
df_chunks = pd.concat(all_chunks, ignore_index=True)

# Para facilitar la unión con train, renombramos la columna 'base_coin' a 'mint'
df_chunks.rename(columns={'base_coin': 'mint'}, inplace=True)

# Ejemplo de estadísticas agregadas: contamos transacciones, sumamos montos,
# calculamos promedios, máximos, etc. para varias columnas.
aggregated = df_chunks.groupby('mint').agg(
    transaction_count = ('mint', 'count'),
    total_base_coin_amount = ('base_coin_amount', 'sum'),
    total_quote_coin_amount = ('quote_coin_amount', 'sum'),
    mean_virtual_token_balance_after = ('virtual_token_balance_after', 'mean'),
    median_virtual_token_balance_after = ('virtual_token_balance_after', 'median'),
    max_virtual_token_balance_after = ('virtual_token_balance_after', 'max'),
    mean_virtual_sol_balance_after = ('virtual_sol_balance_after', 'mean'),
    median_virtual_sol_balance_after = ('virtual_sol_balance_after', 'median'),
    max_virtual_sol_balance_after = ('virtual_sol_balance_after', 'max'),
    total_provided_gas_fee = ('provided_gas_fee', 'sum'),
    mean_fee = ('fee', 'mean'),
    total_consumed_gas = ('consumed_gas', 'sum')
).reset_index()

# También, se puede contar la cantidad de transacciones de cada dirección
# (por ejemplo, cuántas son "buy" y cuántas "sell")
direction_counts = df_chunks.pivot_table(index='mint', 
                                         columns='direction', 
                                         values='tx_idx', 
                                         aggfunc='count').reset_index()
# Renombramos las columnas resultantes para mayor claridad (por ejemplo: buy_count, sell_count)
direction_counts.rename(columns={'buy': 'buy_count', 'sell': 'sell_count'}, inplace=True)

# Unir las estadísticas agregadas con los conteos de dirección
gran_chunk = pd.merge(aggregated, direction_counts, on='mint', how='left')

# Guardar el DataFrame resultante en un archivo CSV
output_path = os.path.join(chunk_dir, "gran_chunk.csv")
gran_chunk.to_csv(output_path, index=False)

print("El archivo 'gran_chunk.csv' ha sido creado y contiene las características agregadas por token.")


El archivo 'gran_chunk.csv' ha sido creado y contiene las características agregadas por token.


GRAN CHUNK V3 : CALCULADA CON TEST Y TRAIN PARA VER SU RESULTADOS

In [21]:
import os
import pandas as pd
import numpy as np

# ——————————————————————————————————
#  Configuración de rutas
# ——————————————————————————————————
base_dir         = r"D:\Desafio Kaggle\Desafios SolanaV2"
chunk_dir        = base_dir
static_path      = os.path.join(base_dir, "gran_chunk.csv")                     # features estáticas
train_path       = os.path.join(base_dir, "train.csv")                           # train con slot_min
test_path        = os.path.join(base_dir, "test_unlabeled.csv")    # test con slot_min
output_path      = os.path.join(base_dir, "gran_chunk_v3.csv")                   # salida

# ——————————————————————————————————
#  1. Cargo estadísticas estáticas
# ——————————————————————————————————
static_feats = pd.read_csv(static_path)

# ——————————————————————————————————
#  2. Cargo slot_min de train y test, y uno ambos
# ——————————————————————————————————
train_sm = pd.read_csv(train_path, usecols=['mint','slot_min'])
test_sm  = pd.read_csv(test_path,  usecols=['mint','slot_min'])
slot_min_all = pd.concat([train_sm, test_sm], ignore_index=True)

# ——————————————————————————————————
#  3. Releo todos los chunks para las dinámicas
# ——————————————————————————————————
all_chunks = []
for i in range(1, 42):
    df = pd.read_csv(os.path.join(chunk_dir, f"chunk_{i}.csv"))
    df.rename(columns={'base_coin': 'mint'}, inplace=True)
    all_chunks.append(df)
df_chunks = pd.concat(all_chunks, ignore_index=True)

# ——————————————————————————————————
#  4. Calculo FEATURES DINÁMICAS
# ——————————————————————————————————
# 4.1 Retraso al primer swap
first_swap = (
    df_chunks.groupby('mint')['slot']
             .min()
             .reset_index(name='first_swap_slot')
)
first_swap = first_swap.merge(slot_min_all, on='mint', how='inner')
first_swap['first_swap_delay'] = (
    first_swap['first_swap_slot'] - first_swap['slot_min']
)

# 4.2 Transacciones por slot (velocidad)
speed = (
    df_chunks.groupby('mint')
             .apply(lambda d: len(d) / (d['slot'].max() - d['slot'].min() + 1))
             .reset_index(name='tx_per_slot')
)

# 4.3 Volatilidad de quote_coin_amount por slot
block_quote = (
    df_chunks.groupby(['mint','slot'])['quote_coin_amount']
             .sum()
             .reset_index()
)
vol = (
    block_quote.groupby('mint')['quote_coin_amount']
               .std()
               .reset_index(name='quote_volatility')
)

# 4.4 Concentración de wallets (top-3 / total)
wallet_sum = (
    df_chunks.groupby(['mint','signing_wallet'])['quote_coin_amount']
             .sum()
             .reset_index()
)
wallet_conc = (
    wallet_sum.groupby('mint')
              .apply(lambda d: d.nlargest(3, 'quote_coin_amount')['quote_coin_amount'].sum()
                           / d['quote_coin_amount'].sum())
              .reset_index(name='wallet_concentration')
)

# Empaquetar todas en un único DF
dynamic_feats = (
    first_swap[['mint','first_swap_delay']]
    .merge(speed,       on='mint', how='left')
    .merge(vol,         on='mint', how='left')
    .merge(wallet_conc, on='mint', how='left')
)

# ——————————————————————————————————
#  5. Merge con estáticas y relleno de NaNs
# ——————————————————————————————————
gran_chunk_v3 = static_feats.merge(dynamic_feats, on='mint', how='left')
for c in ['first_swap_delay','tx_per_slot','quote_volatility','wallet_concentration']:
    gran_chunk_v3[c] = gran_chunk_v3[c].fillna(0)

# ——————————————————————————————————
#  6. Grabo el resultado
# ——————————————————————————————————
gran_chunk_v3.to_csv(output_path, index=False)
print("gran_chunk_v3.csv creado con columnas:", gran_chunk_v3.columns.tolist())


C:\Users\lenovo\AppData\Local\Temp\ipykernel_24336\2195756955.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_chunks.groupby('mint')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_24336\2195756955.py:78: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda d: d.nlargest(3, 'quote_coin_amount')['quote_coin_amount'].sum()
C:\Users\lenovo\AppData\Local\Temp\ipykernel_24336\2195756955.py:78: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda d: d.nlargest(3, 'quote_coin_amount')['quote_coin_amount'].sum()
C:\Users\lenovo\AppData\Local\Temp\ipykernel_24336\2195756955.py:78: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda d:

gran_chunk_v3.csv creado con columnas: ['mint', 'transaction_count', 'total_base_coin_amount', 'total_quote_coin_amount', 'mean_virtual_token_balance_after', 'median_virtual_token_balance_after', 'max_virtual_token_balance_after', 'mean_virtual_sol_balance_after', 'median_virtual_sol_balance_after', 'max_virtual_sol_balance_after', 'total_provided_gas_fee', 'mean_fee', 'total_consumed_gas', 'buy_count', 'sell_count', 'first_swap_delay', 'tx_per_slot', 'quote_volatility', 'wallet_concentration']


In [22]:
import os
import pandas as pd

# Define la ruta base donde se encuentran todos los archivos
base_dir = r"D:\Desafio Kaggle\Desafios SolanaV2"

# Rutas de los archivos
gran_chunk_path = os.path.join(base_dir, "gran_chunk_v3.csv")
train_path = os.path.join(base_dir, "train.csv")
test_unlabeled_path = os.path.join(base_dir, "test_unlabeled.csv")

# Cargar gran_chunk.csv
gran_chunk = pd.read_csv(gran_chunk_path)

# Cargar train.csv
train_df = pd.read_csv(train_path)

# Unir gran_chunk con train usando la columna "mint"
gran_train = pd.merge(train_df, gran_chunk, on="mint", how="left")

# Guardar el DataFrame resultante en "gran_train.csv"
gran_train_output_path = os.path.join(base_dir, "gran_train.csv")
gran_train.to_csv(gran_train_output_path, index=False)

# Cargar test_unlabeled.csv
test_unlabeled = pd.read_csv(test_unlabeled_path)

# Unir gran_chunk con test_unlabeled usando la columna "mint"
gran_test_unlabeled = pd.merge(test_unlabeled, gran_chunk, on="mint", how="left")

# Guardar el DataFrame resultante en "gran_test_unlabeled.csv"
gran_test_unlabeled_output_path = os.path.join(base_dir, "gran_test_unlabeled.csv")
gran_test_unlabeled.to_csv(gran_test_unlabeled_output_path, index=False)

print("Merge completado. Se han creado los archivos gran_train.csv y gran_test_unlabeled.csv")


Merge completado. Se han creado los archivos gran_train.csv y gran_test_unlabeled.csv


In [23]:
import os
import pandas as pd

# Define la ruta base
base_dir = r"D:\Desafio Kaggle\Desafios SolanaV2"

# Rutas de los archivos ya creados
gran_train_path = os.path.join(base_dir, "gran_train.csv")
gran_test_unlabeled_path = os.path.join(base_dir, "gran_test_unlabeled.csv")

# Cargar los conjuntos base
gran_train = pd.read_csv(gran_train_path)
gran_test_unlabeled = pd.read_csv(gran_test_unlabeled_path)

# -------------------------------
# Procesar el archivo dune_token_info
# -------------------------------
dune_info_path = os.path.join(base_dir, "dune_token_info.csv")
dune_info = pd.read_csv(dune_info_path)

# Seleccionar las columnas importantes y renombrar la llave
dune_info = dune_info[['token_mint_address', 'decimals', 'created_at']]
dune_info.rename(columns={'token_mint_address': 'mint'}, inplace=True)

# -------------------------------
# Procesar el archivo token_info_onchain_drivers
# -------------------------------
onchain_drivers_path = os.path.join(base_dir, "token_info_onchain_divers.csv")
onchain_drivers = pd.read_csv(onchain_drivers_path)

# Seleccionar las columnas importantes
onchain_drivers = onchain_drivers[['mint', 'creator', 'bundle_size', 'gas_used', 'bundled_buys_count', 'dev_balance']]

# -------------------------------
# Enriquecer gran_train
# -------------------------------
# Merge con dune_info
gran_train_enriched = pd.merge(gran_train, dune_info, on='mint', how='left')
# Merge con onchain_drivers
gran_train_enriched = pd.merge(gran_train_enriched, onchain_drivers, on='mint', how='left')

# Guardar el archivo enriquecido para el conjunto de train
gran_train_enriched_path = os.path.join(base_dir, "gran_train_enriched.csv")
gran_train_enriched.to_csv(gran_train_enriched_path, index=False)

# -------------------------------
# Enriquecer gran_test_unlabeled
# -------------------------------
# Merge con dune_info
gran_test_unlabeled_enriched = pd.merge(gran_test_unlabeled, dune_info, on='mint', how='left')
# Merge con onchain_drivers
gran_test_unlabeled_enriched = pd.merge(gran_test_unlabeled_enriched, onchain_drivers, on='mint', how='left')

# Guardar el archivo enriquecido para el conjunto de test
gran_test_unlabeled_enriched_path = os.path.join(base_dir, "gran_test_unlabeled_enriched.csv")
gran_test_unlabeled_enriched.to_csv(gran_test_unlabeled_enriched_path, index=False)

print("Enriquecimiento completado. Se han creado 'gran_train_enriched.csv' y 'gran_test_unlabeled_enriched.csv'.")


C:\Users\lenovo\AppData\Local\Temp\ipykernel_24336\3007575454.py:29: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  onchain_drivers = pd.read_csv(onchain_drivers_path)


Enriquecimiento completado. Se han creado 'gran_train_enriched.csv' y 'gran_test_unlabeled_enriched.csv'.


In [24]:
import os
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

# 1. Rutas
base_dir    = r"D:\Desafio Kaggle\Desafios SolanaV2"
train_in    = os.path.join(base_dir, "gran_train_enriched.csv")
test_in     = os.path.join(base_dir, "gran_test_unlabeled_enriched.csv")
orig_labels = os.path.join(base_dir, "train.csv")
train_out   = os.path.join(base_dir, "gran_train_enriched_final.csv")
test_out    = os.path.join(base_dir, "gran_test_unlabeled_enriched_final.csv")

# 2. Cargo
train = pd.read_csv(train_in, low_memory=False)
test  = pd.read_csv(test_in,  low_memory=False)

# 3. Flags de missingness
flag_cols = ['buy_count','sell_count','decimals','created_at',
             'creator','bundle_size','gas_used','bundled_buys_count','dev_balance']
for c in flag_cols:
    train[f"is_{c}_missing"] = train[c].isna().astype(int)
    test [f"is_{c}_missing"] = test [c].isna().astype(int)

# 4. Timestamp numeric de created_at
for df in (train, test):
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    df['created_ts'] = df['created_at'].astype(np.int64) // 10**9
    df.drop(columns=['created_at'], inplace=True)

# 5. Winsorización de gas_used
p1, p99 = train['gas_used'].quantile([0.01, 0.99])
for df in (train, test):
    df['gas_used'] = df['gas_used'].clip(p1, p99)

# 6. Imputación por grupos en bundle_size y decimals
train['txn_bin'] = pd.qcut(train['transaction_count'], 5, labels=False, duplicates='drop')
test ['txn_bin'] = pd.qcut(test ['transaction_count'], 5, labels=False, duplicates='drop')
for col in ['bundle_size','decimals']:
    med = train.groupby('txn_bin')[col].median()
    train[col] = train[col].fillna(train['txn_bin'].map(med))
    test [col] = test [col].fillna(test ['txn_bin'].map(med))

# 7. Merge de etiqueta para target-encoding
labels = pd.read_csv(orig_labels, usecols=['mint','has_graduated'])
train  = train.merge(labels, on='mint', how='left')
print(train.columns.tolist())
# Después de hacer el merge:
# 7.5) Normalizar la columna de la etiqueta para que se llame 'has_graduated'
train.rename(columns={'has_graduated_y':'has_graduated'}, inplace=True)
# y eliminar la copia antigua
train.drop(columns=['has_graduated_x'], inplace=True)


# 8. Target-encoding manual de creator
k = 10
gm = train['has_graduated'].mean()

agg = train.groupby('creator')['has_graduated'] \
           .agg(['mean','count']).rename(columns={'mean':'m','count':'n'})
agg['te'] = (agg['m']*agg['n'] + gm*k) / (agg['n'] + k)
map_te = agg['te'].to_dict()

train['creator_te'] = train['creator'].map(map_te).fillna(gm)
test ['creator_te'] = test ['creator'].map(map_te).fillna(gm)

# 9. Imputación iterativa multivariada
# ---------------------------------------------------
# 9. Imputación iterativa multivariada (versión corregida)
# ---------------------------------------------------
# columnas numéricas en train (sin etiqueta ni slot_graduated)
num_cols = (
    train.select_dtypes(include=[np.number])
         .columns
         .difference(['has_graduated','slot_graduated'])
)

# columnas numéricas que efectivamente están en test
common_cols = [c for c in num_cols if c in test.columns]

imp = IterativeImputer(random_state=42, max_iter=10, initial_strategy='median')
train[common_cols] = imp.fit_transform(train[common_cols])
test [common_cols] = imp.transform(test [common_cols])

# 10. Limpieza final y guardado
train.drop(columns=['has_graduated','creator','txn_bin'], errors='ignore', inplace=True)
test .drop(columns=['creator','txn_bin'], errors='ignore', inplace=True)

train.to_csv(train_out, index=False)
test .to_csv(test_out,  index=False)

print("Hecho. Archivos finales:")
print(" - Train:", train_out)
print(" - Test :", test_out)



['mint', 'slot_min', 'slot_graduated', 'has_graduated_x', 'transaction_count', 'total_base_coin_amount', 'total_quote_coin_amount', 'mean_virtual_token_balance_after', 'median_virtual_token_balance_after', 'max_virtual_token_balance_after', 'mean_virtual_sol_balance_after', 'median_virtual_sol_balance_after', 'max_virtual_sol_balance_after', 'total_provided_gas_fee', 'mean_fee', 'total_consumed_gas', 'buy_count', 'sell_count', 'first_swap_delay', 'tx_per_slot', 'quote_volatility', 'wallet_concentration', 'decimals', 'creator', 'bundle_size', 'gas_used', 'bundled_buys_count', 'dev_balance', 'is_buy_count_missing', 'is_sell_count_missing', 'is_decimals_missing', 'is_created_at_missing', 'is_creator_missing', 'is_bundle_size_missing', 'is_gas_used_missing', 'is_bundled_buys_count_missing', 'is_dev_balance_missing', 'created_ts', 'txn_bin', 'has_graduated_y']
Hecho. Archivos finales:
 - Train: D:\Desafio Kaggle\Desafios SolanaV2\gran_train_enriched_final.csv
 - Test : D:\Desafio Kaggle\Des

In [25]:
import pandas as pd 

df=pd.read_csv(train_out)
df1=pd.read_csv(test_out)
print(df.columns.tolist())
print(df1.columns.tolist())

['mint', 'slot_min', 'slot_graduated', 'transaction_count', 'total_base_coin_amount', 'total_quote_coin_amount', 'mean_virtual_token_balance_after', 'median_virtual_token_balance_after', 'max_virtual_token_balance_after', 'mean_virtual_sol_balance_after', 'median_virtual_sol_balance_after', 'max_virtual_sol_balance_after', 'total_provided_gas_fee', 'mean_fee', 'total_consumed_gas', 'buy_count', 'sell_count', 'first_swap_delay', 'tx_per_slot', 'quote_volatility', 'wallet_concentration', 'decimals', 'bundle_size', 'gas_used', 'bundled_buys_count', 'dev_balance', 'is_buy_count_missing', 'is_sell_count_missing', 'is_decimals_missing', 'is_created_at_missing', 'is_creator_missing', 'is_bundle_size_missing', 'is_gas_used_missing', 'is_bundled_buys_count_missing', 'is_dev_balance_missing', 'created_ts', 'creator_te']
['mint', 'slot_min', 'transaction_count', 'total_base_coin_amount', 'total_quote_coin_amount', 'mean_virtual_token_balance_after', 'median_virtual_token_balance_after', 'max_virt

In [9]:
a=len(train_out)
b=len(test_out)
print(a,b)

65 74


In [17]:
import pandas as pd

df_train = pd.read_csv(train_out)
df_test  = pd.read_csv(test_out)

train_cols = set(df_train.columns)
test_cols  = set(df_test.columns)

# Columnas que están en train pero faltan en test
missing_in_test = train_cols - test_cols
print("En train y NO en test:", missing_in_test)

# Columnas que están en test pero faltan en train
missing_in_train = test_cols - train_cols
print("En test y NO en train:", missing_in_train)


En train y NO en test: {'slot_graduated'}
En test y NO en train: set()


Entrenamiento del modelo con dune token info y onchain drivers v2

In [26]:
import os
import joblib
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

# 1. Rutas
base_dir       = r"D:\Desafio Kaggle\Desafios SolanaV2"
train_path     = os.path.join(base_dir, "gran_train_enriched_final.csv")
labels_path    = os.path.join(base_dir, "train.csv")
dune_v2_path   = os.path.join(base_dir, "dune_token_info_v2.csv")
onchain_v2_path= os.path.join(base_dir, "token_info_onchain_divers_v2.csv")

# 2. Cargo el dataset enriquecido
df = pd.read_csv(train_path, low_memory=False)

# 3. Creo las features de los V2 y hago merge (igual que antes)
# 3.1 Dune v2
dune_v2 = pd.read_csv(dune_v2_path, low_memory=False) \
           .rename(columns={"token_mint_address":"mint"})
dune_v2['created_at'] = pd.to_datetime(dune_v2['created_at'], errors='coerce')
dune_feats = pd.DataFrame({
    'mint':        dune_v2['mint'],
    'dune_decimals':   dune_v2['decimals'].fillna(6),
    'dune_name_len':   dune_v2['name'].fillna("").str.len(),
    'dune_symbol_len': dune_v2['symbol'].fillna("").str.len(),
    'dune_hour':       dune_v2['created_at'].dt.hour.fillna(0).astype(int),
    'dune_weekday':    dune_v2['created_at'].dt.weekday.fillna(0).astype(int),
})

# 3.2 Onchain v2
onchain_v2 = pd.read_csv(onchain_v2_path, low_memory=False)
onchain_v2['block_time'] = pd.to_datetime(onchain_v2['block_time'], errors='coerce')
creator_freq = onchain_v2['creator'].value_counts(normalize=True)
version_freq = onchain_v2['version'].value_counts(normalize=True)
onch_feats = pd.DataFrame({
    'mint':               onchain_v2['mint'],
    'onch_bundle_size':   onchain_v2['bundle_size'].fillna(1),
    'onch_gas_used':      onchain_v2['gas_used'].fillna(onchain_v2['gas_used'].median()),
    'onch_instr':         onchain_v2['amount_of_instructions'].fillna(0),
    'onch_reads':         onchain_v2['amount_of_lookup_reads'].fillna(0),
    'onch_writes':        onchain_v2['amount_of_lookup_writes'].fillna(0),
    'onch_creator_freq':  onchain_v2['creator'].map(creator_freq).fillna(0),
    'onch_version_freq':  onchain_v2['version'].map(version_freq).fillna(0),
    'onch_direct_pf':     onchain_v2['direct_pf_invocation'].fillna(0),
    'onch_hour':          onchain_v2['block_time'].dt.hour.fillna(0).astype(int),
    'onch_weekday':       onchain_v2['block_time'].dt.weekday.fillna(0).astype(int),
})

data = df.merge(dune_feats, on='mint', how='left') \
         .merge(onch_feats, on='mint', how='left')

# 4. Relleno NaNs en las nuevas features
for col in dune_feats.columns.difference(['mint']).tolist() + onch_feats.columns.difference(['mint']).tolist():
    data[col] = data[col].fillna(0)

# 5. Incorporo la etiqueta original (Option B)
labels = pd.read_csv(labels_path, usecols=['mint','has_graduated'])
data   = data.merge(labels, on='mint', how='left')

# 7. Preparo X e y
drop_cols = ['mint','creator','slot_graduated']  # elimino identificadores y slot_graduated si no la uso
X = data.drop(columns=drop_cols + ['has_graduated'], errors='ignore')
y = data['has_graduated'].astype(int)

# 8. Split train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 9. Entreno LightGBM
best_params = {
    'objective':'binary','metric':'binary_logloss','boosting_type':'gbdt',
    'learning_rate':0.02,'num_leaves':255,'min_data_in_leaf':5,
    'feature_fraction':0.4,'bagging_fraction':0.8,'bagging_freq':5,
    'lambda_l1':0.1,'lambda_l2':0.2,'max_depth':9,'min_split_gain':0.1,
    'max_bin':255,'verbose':-1,'seed':42
}
train_set = lgb.Dataset(X_train, label=y_train)
val_set   = lgb.Dataset(X_val,   label=y_val, reference=train_set)

model = lgb.train(
    best_params,
    train_set,
    num_boost_round=2000,
    valid_sets=[val_set],
    callbacks=[ lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=0) ]
)

# 10. Evalúo
y_pred = model.predict(X_val, num_iteration=model.best_iteration)
print("Log Loss en validación:", log_loss(y_val, y_pred))

# 11. Guardo
out_path = os.path.join(base_dir, "best_model_with_v1_features.pkl")
joblib.dump(model, out_path)
print("Modelo guardado en:", out_path)


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[509]	valid_0's binary_logloss: 0.016184
Log Loss en validación: 0.016183993239336746
Modelo guardado en: D:\Desafio Kaggle\Desafios SolanaV2\best_model_with_v1_features.pkl


In [27]:
import os
import joblib
import pandas as pd
import numpy as np

# Paths
base_dir        = r"D:\Desafio Kaggle\Desafios SolanaV2"
model_path      = os.path.join(base_dir, "best_model_with_v1_features.pkl")
test_path       = os.path.join(base_dir, "gran_test_unlabeled_enriched_final.csv")
dune_v2_path    = os.path.join(base_dir, "dune_token_info_v2.csv")
onchain_v2_path = os.path.join(base_dir, "token_info_onchain_divers_v2.csv")

# 1. Cargar modelo
model = joblib.load(model_path)

# 2. Leer test_unlabeled enriquecido
test_df = pd.read_csv(test_path, low_memory=False)

# 3. Crear features desde v2

# 3.1 Dune v2
dune_v2 = (
    pd.read_csv(dune_v2_path, low_memory=False)
      .rename(columns={"token_mint_address":"mint"})
)
dune_v2["created_at"] = pd.to_datetime(dune_v2["created_at"], errors="coerce")
dune_feats = pd.DataFrame({
    "mint":           dune_v2["mint"],
    "dune_decimals":  dune_v2["decimals"].fillna(6),
    "dune_name_len":  dune_v2["name"].fillna("").str.len(),
    "dune_symbol_len":dune_v2["symbol"].fillna("").str.len(),
    "dune_hour":      dune_v2["created_at"].dt.hour.fillna(0).astype(int),
    "dune_weekday":   dune_v2["created_at"].dt.weekday.fillna(0).astype(int),
})

# 3.2 Onchain v2
onchain_v2 = pd.read_csv(onchain_v2_path, low_memory=False)
onchain_v2["block_time"] = pd.to_datetime(onchain_v2["block_time"], errors="coerce")
creator_freq = onchain_v2["creator"].value_counts(normalize=True)
version_freq = onchain_v2["version"].value_counts(normalize=True)
onch_feats = pd.DataFrame({
    "mint":              onchain_v2["mint"],
    "onch_bundle_size":  onchain_v2["bundle_size"].fillna(1),
    "onch_gas_used":     onchain_v2["gas_used"].fillna(onchain_v2["gas_used"].median()),
    "onch_instr":        onchain_v2["amount_of_instructions"].fillna(0),
    "onch_reads":        onchain_v2["amount_of_lookup_reads"].fillna(0),
    "onch_writes":       onchain_v2["amount_of_lookup_writes"].fillna(0),
    "onch_creator_freq": onchain_v2["creator"].map(creator_freq).fillna(0),
    "onch_version_freq": onchain_v2["version"].map(version_freq).fillna(0),
    "onch_direct_pf":    onchain_v2["direct_pf_invocation"].fillna(0),
    "onch_hour":         onchain_v2["block_time"].dt.hour.fillna(0).astype(int),
    "onch_weekday":      onchain_v2["block_time"].dt.weekday.fillna(0).astype(int),
})

# 4. Merge las nuevas features
test_merged = (
    test_df
      .merge(dune_feats, on="mint", how="left")
      .merge(onch_feats, on="mint", how="left")
)

# 5. Rellenar NaNs de las nuevas columnas
for col in dune_feats.columns.difference(["mint"]).tolist() + onch_feats.columns.difference(["mint"]).tolist():
    test_merged[col] = test_merged[col].fillna(0)

# 6. Alinear test con las features del modelo
feature_cols = model.feature_name()
X_test = test_merged[feature_cols]

# 7. Predecir probabilidades
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# 8. Crear submission
submission = pd.DataFrame({
    "mint":           test_merged["mint"],
    "has_graduated":  y_pred
})

submission_path = os.path.join(base_dir, "submission_v3.csv")
submission.to_csv(submission_path, index=False)

# 9. Vista previa
print("Submission preview:")
print(submission.head())
print(f"\nSaved submission to: {submission_path}")


Submission preview:
                                           mint  has_graduated
0  9Wt3N7etKMX9cioTdEJ5S4b8A9nK3M66n9RFVgBGpump       0.001420
1  9q5y2X2P8ZEKTjyXBVcS5q2EZM7HbNV8DURY2qnvqi2f       0.000645
2  HL2di8dcQ7eYDmkcFoZ4zJyHX5SbRZXAJxTegL3JPfx2       0.000412
3  7iAFj9Pc5QH9jbGmHwYe8T6yzNVbjhL13PNJXVTspump       0.004028
4  F7U1Rdgz2KFpneKpAnYytWF2jggnsrLScfi2A668pump       0.001940

Saved submission to: D:\Desafio Kaggle\Desafios SolanaV2\submission_v3.csv


In [29]:
import pandas as pd

# 1. Carga tu submission original
df = pd.read_csv(r"D:\Desafio Kaggle\Desafios SolanaV2\submission_v3.csv")

# 2. Elimina duplicados por 'mint', manteniendo la primera ocurrencia
df_clean = df.drop_duplicates(subset="mint", keep="first")

# 3. (Opcional) Verifica el conteo de filas
print("Filas antes:", len(df))
print("Filas después:", len(df_clean))

# 4. Guarda el CSV corregido
df_clean.to_csv(r"D:\Desafio Kaggle\Desafios SolanaV2\submission_fixed.csv", index=False)


Filas antes: 478834
Filas después: 478832
